In [2]:
import tensorflow as tf
import pandas as pd
import requests 
import json
import base64

from modules.utils import create_int_feature, create_float_feature, create_str_feature
from typing import Dict

In [3]:
def prepare_json(inputs: Dict):
    feature_spec = {}
    
    for key, value in inputs.items():
        if isinstance(value, float):
            feature_spec[key] = create_float_feature(value)
        elif isinstance(value, str):
            feature_spec[key] = create_str_feature(value)
        else:    
            feature_spec[key] = create_int_feature(value)
        
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({"instances": result})

In [7]:
def make_prediction(inputs: Dict):
    json_data = prepare_json(inputs)
    
    endpoint = "http://localhost:8080/v1/models/movies-cbf-model:predict"
    response = requests.post(endpoint, data=json_data)
    
    prediction = response.json()["predictions"][0][0]
    
    return f"Predict movie rating with Movie Id {inputs['movieId']} and User Id {inputs['userId']}: {prediction}"

In [27]:
make_prediction({"movieId": 2, "userId": 2, "genres": "Romance"})


'Predict movie rating with Movie Id 2 and User Id 2: 0.999995828'